In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
# Set the path to the directory where the data is stored
data_dir = 'content/drive/MyDrive/m5-forecasting-uncertainty/Vegetables/'

# Initial Model

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define the parameters for the data pre-processing
batch_size = 32
img_height = 180
img_width = 180

# Create a data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of the images for validation
    # Add any other data augmentation parameters you might need
)

# Create a data generator for validation data
# Note: No need for data augmentation here
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # because we have multiple classes
    subset='training',  # set as training data
    shuffle=True
)

# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # because we have multiple classes
    subset='validation',  # set as validation data
    shuffle=False  # important for correct label matching
)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512),
    Activation('relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes),
    Activation('softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the model
model.save('product_categorization_model.h5')

Found 514 images belonging to 15 classes.
Found 120 images belonging to 15 classes.


Epoch 1/50
16/16 [==============================] - 250s 16s/step - loss: 2.7084 - accuracy: 0.1743 - val_loss: 2.4320 - val_accuracy: 0.2708
Epoch 2/50
16/16 [==============================] - 12s 723ms/step - loss: 1.9385 - accuracy: 0.3278 - val_loss: 2.3554 - val_accuracy: 0.1667
Epoch 3/50
16/16 [==============================] - 12s 726ms/step - loss: 1.6766 - accuracy: 0.4315 - val_loss: 1.9404 - val_accuracy: 0.4271
Epoch 4/50
16/16 [==============================] - 12s 713ms/step - loss: 1.5016 - accuracy: 0.5041 - val_loss: 1.7409 - val_accuracy: 0.4062
Epoch 5/50
16/16 [==============================] - 11s 750ms/step - loss: 1.3216 - accuracy: 0.5498 - val_loss: 1.3964 - val_accuracy: 0.4792
Epoch 6/50
16/16 [==============================] - 12s 706ms/step - loss: 0.8429 - accuracy: 0.6950 - val_loss: 1.2988 - val_accuracy: 0.5625
Epoch 7/50
16/16 [==============================] - 12s 712ms/step - loss: 0.6286 - accuracy: 0.7946 - val_loss: 1.0698 - val_accuracy: 0.6250


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // batch_size
)

print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')

3/3 [==============================] - 1s 257ms/step - loss: 3.5389 - accuracy: 0.3750
Validation Loss: 3.538930892944336
Validation Accuracy: 0.375


In [13]:
# Print the Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)

[[ 2  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  1  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 1  0  2  0  0  0  0  0  0  0  1  0  0  2  1]
 [ 1  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  4  0  0  0  0]
 [ 0  0  0  0  1  4  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  3  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  1  1  0  0  1  0  0  0]
 [ 1  0  0  0  0  0  0  0  2  0  0  0  1  0  0]
 [ 1  0  0  0  0  1  0  0  0  3  0  1  0  1  0]
 [ 1  0  0  0  0  4  0  0  0  0  0  0  2  1 14]
 [ 0  0  1  0  0  0  0  0  2  3  0  8  1  0  0]
 [ 0  1  0  0  0  0  0  0  1  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  5  0  0 18  2]
 [ 0  0  1  0  0  1  0  0  0  0  0  0  0  0  4]]


# Final Model

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight



# Define the parameters for the data pre-processing
batch_size = 32
img_height = 224  # MobileNetV2 default size
img_width = 224   # MobileNetV2 default size

# Create a data generator for training data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a data generator for validation data (no augmentation here)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Load MobileNetV2 pre-trained model
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze the layers

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Calculate class weights for imbalanced datasets
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes)
class_weight_dict = {i : class_weights[i] for i in range(len(class_weights))}

# Train the model with class weights and callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    class_weight=class_weight_dict,
    callbacks=[early_stop, checkpoint]
)

# Save the model
model.save('product_categorization_model.h5')

# Evaluate the model
eval_result = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Validation Loss: {eval_result[0]}')
print(f'Validation Accuracy: {eval_result[1]}')


Found 514 images belonging to 15 classes.
Found 120 images belonging to 15 classes.
Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 2.1889 - accuracy: 0.4627

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - 17s 859ms/step - loss: 2.1889 - accuracy: 0.4627 - val_loss: 0.7991 - val_accuracy: 0.7500
Epoch 2/50
16/16 [==============================] - 13s 815ms/step - loss: 0.7695 - accuracy: 0.7910 - val_loss: 0.3885 - val_accuracy: 0.8854
Epoch 3/50
16/16 [==============================] - 13s 771ms/step - loss: 0.4653 - accuracy: 0.8589 - val_loss: 0.2584 - val_accuracy: 0.9271
Epoch 4/50
16/16 [==============================] - 12s 756ms/step - loss: 0.3870 - accuracy: 0.8817 - val_loss: 0.2612 - val_accuracy: 0.9375
Epoch 5/50
16/16 [==============================] - 12s 765ms/step - loss: 0.2971 - accuracy: 0.9025 - val_loss: 0.1875 - val_accuracy: 0.9375
Epoch 6/50
16/16 [==============================] - 12s 828ms/step - loss: 0.1795 - accuracy: 0.9398 - val_loss: 0.1810 - val_accuracy: 0.9375
Epoch 7/50
16/16 [==============================] - 12s 760ms/step - loss: 0.1728 - accuracy: 0.9419 - val_loss: 0.1245 - val_accuracy: 0.9583
Epoch 8/50

In [18]:
# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // batch_size
)

print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')

3/3 [==============================] - 2s 470ms/step - loss: 0.1026 - accuracy: 0.9479
Validation Loss: 0.10263439267873764
Validation Accuracy: 0.9479166865348816


In [19]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from math import ceil

# Make sure the generator is not shuffling the data
validation_generator.shuffle = False

# Reset the validation generator before making predictions
validation_generator.reset()

# Predict on the entire validation set
predictions = model.predict(validation_generator, steps=ceil(validation_generator.samples / batch_size))
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = validation_generator.classes

# Make sure the predictions and true labels have the same length
predicted_classes = predicted_classes[:len(true_classes)]

# Get the class labels
class_labels = list(validation_generator.class_indices.keys())

# Print the Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)

4/4 [==============================] - 3s 357ms/step
                    precision    recall  f1-score   support

         Asparagus       1.00      1.00      1.00         3
         Aubergine       0.80      1.00      0.89         4
Brown-Cap-Mushroom       0.86      0.86      0.86         7
           Cabbage       1.00      1.00      1.00         3
           Carrots       1.00      1.00      1.00         8
          Cucumber       1.00      1.00      1.00         5
            Garlic       1.00      1.00      1.00         5
            Ginger       1.00      1.00      1.00         3
              Leek       1.00      1.00      1.00         4
             Onion       1.00      0.71      0.83         7
            Pepper       1.00      0.91      0.95        22
            Potato       0.94      1.00      0.97        15
          Red-Beet       0.75      1.00      0.86         3
            Tomato       1.00      1.00      1.00        25
          Zucchini       0.86      1.00      0